# Slow Fusion

This script takes all frames at once and passes them through a cnn.

In [35]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, Conv3D, MaxPooling2D, MaxPooling3D, LayerNormalization
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import initializers
import time
import numpy as np

## 1. Load Data

To begin, data for a middle frames model must be loaded along with the expected regional rainfall values.

### 1.0 New Training Data

This section loads the new training data set.

In [2]:
training_datafile = "D:/PHD_DATA/Video_18-01-2021/prepared-data/middle_all.npy"
training_rainfallfile = "D:/PHD_DATA/Video_18-01-2021/prepared-data/expected_all.npy"

In [3]:
training_videos = np.load(training_datafile)
training_rainfall = np.load(training_rainfallfile)[:, 2:]

In [4]:
training_videos = np.swapaxes(training_videos, 1, 2)
training_videos = np.swapaxes(training_videos, 2, 3)
training_videos = np.swapaxes(training_videos, 3, 4)

In [5]:
training_videos[:, :, :, 0] = (training_videos[:, :, :, 0] - np.min(training_videos[:, :, :, 0])) / (np.max(training_videos[:, :, :, 0]) - np.min(training_videos[:, :, :, 0]))
training_videos[:, :, :, 1] = (training_videos[:, :, :, 1] - np.min(training_videos[:, :, :, 1])) / (np.max(training_videos[:, :, :, 1]) - np.min(training_videos[:, :, :, 1]))

#training_videos[:, :, :, 0] = (training_videos[:, :, :, 0] - np.mean(training_videos[:, :, :, 0])) / np.std(training_videos[:, :, :, 0])
#training_videos[:, :, :, 1] = (training_videos[:, :, :, 1] - np.mean(training_videos[:, :, :, 1])) / np.std(training_videos[:, :, :, 1]) 

### 1.1 DEPRECATED

This following section uses two separate datasets. Which is not required.

In [6]:
#training_datafile = "E:/31-12-2020/prepared-data/middle_train.npy"
#validation_datafile = "E:/31-12-2020/prepared-data/middle_valid.npy"
#
#training_rainfallfile = "E:/31-12-2020/prepared-data/expected_train.npy"
#validation_rainfallfile = "E:/31-12-2020/prepared-data/expected_valid.npy"

In [7]:
#training_videos = np.load(training_datafile)
#validation_videos = np.load(validation_datafile)
#
#training_rainfall = np.load(training_rainfallfile)[:, 2:]
#validation_rainfall = np.load(validation_rainfallfile)[:, 2:]

In [8]:
# We need them in X, Y, COLOURS
#training_videos = np.swapaxes(training_videos, 1, 2)
#training_videos = np.swapaxes(training_videos, 2, 3)
#training_videos = np.swapaxes(training_videos, 3, 4)
#
#validation_videos = np.swapaxes(validation_videos, 1, 2)
#validation_videos = np.swapaxes(validation_videos, 2, 3)
#validation_videos = np.swapaxes(validation_videos, 3, 4)

In [9]:
# Scale the data
#training_videos[:, :, :, :, 0] = (training_videos[:, :, :, :, 0] - np.min(training_videos[:, :, :, :, 0])) / (np.max(training_videos[:, :, :, :, 0]) - np.min(training_videos[:, :, :, :, 0]))
#training_videos[:, :, :, :, 1] = (training_videos[:, :, :, :, 1] - np.min(training_videos[:, :, :, :, 1])) / (np.max(training_videos[:, :, :, :, 1]) - np.min(training_videos[:, :, :, :, 1]))

#validation_videos[:, :, :, :, 0] = (validation_videos[:, :, :, :, 0] - np.min(validation_videos[:, :, :, :, 0])) / (np.max(validation_videos[:, :, :, :, 0]) - np.min(validation_videos[:, :, :, :, 0]))
#validation_videos[:, :, :, :, 1] = (validation_videos[:, :, :, :, 1] - np.min(validation_videos[:, :, :, :, 1])) / (np.max(validation_videos[:, :, :, :, 1]) - np.min(validation_videos[:, :, :, :, 1]))

## 2. Model Definition

Next, a CNN model architecture is defined.

In [29]:
class SlowFusion(Model):
    
    def __init__(self):
        super(SlowFusion, self).__init__()
        
        # First level
        self._first_layers = [
            self._generate_conv_layer(32, (2, 2, 2), (7, 61, 121, 2)),
            self._generate_conv_layer(32, (2, 2, 2), (7, 61, 121, 2)),
            self._generate_conv_layer(32, (2, 2, 2), (7, 61, 121, 2)),
            self._generate_conv_layer(32, (2, 2, 2), (7, 61, 121, 2))
        ]
        
        # Second level
        self._second_layers = [
            self._generate_conv_layer(16, (2, 2, 2), (6, 30, 60, 32)),
            self._generate_conv_layer(16, (2, 2, 2), (6, 30, 60, 32))
        ]
        
        # Third level
        self._third_layers = [
            self._generate_conv_layer(8, (2, 2, 2), (4, 14, 29, 16))
        ]
        
        # Final Dense layer
        self._final_layer = self._generate_dense_layer(13)
    
    def call(self, inputs):
        
        outputs = self._layer_one(inputs)
        #print(outputs[0].shape)
        outputs = self._layer_two(outputs)
        #print(outputs[0].shape)
        outputs = self._layer_three(outputs)
        #print(outputs.shape)
        outputs = self._final_layer(outputs)
        #print(outputs.shape)
        return outputs
        
    def _layer_one(self, inputs):
        outputs = []
        for n in range(0, 4):
            outputs.append(
                self._first_layers[n](
                    inputs[:, (n*7):((n+1)*7), :, :, :]
                )
            )
        trans_outputs = [
            tf.concat((outputs[0], outputs[1]), axis=1),
            tf.concat((outputs[2], outputs[3]), axis=1)
        ]
        return trans_outputs
    
    def _layer_two(self, inputs):
        outputs = []
        for n in range(0, 2):
            outputs.append(self._second_layers[n](inputs[n]))
        trans_outputs = [
            tf.concat((outputs[0], outputs[1]), axis=1)
        ]
        return trans_outputs
    
    def _layer_three(self, inputs):
        return self._third_layers[0](inputs)
    
    def _generate_conv_layer(self, filters, poolsize, input_shape):
        layer = Sequential()
        layer.add(Conv3D(
            filters, poolsize, input_shape=input_shape,
            kernel_initializer=initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)))
        layer.add(Activation('relu'))
        #layer.add(LayerNormalization())
        layer.add(MaxPooling3D(pool_size=poolsize))
        return layer
    
    def _generate_dense_layer(self, output_size):
        layer = Sequential()
        layer.add(Flatten())
        layer.add(Dense(output_size, kernel_initializer=initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)))
        layer.add(Activation('relu'))
        return layer

In [30]:
def model_generator(input_shape=(28, 61, 121, 2), learning_rate=0.1):
    """ This method generates a model definition. """
    model = SlowFusion()
    
    # Setup training mechanism
    model.compile(
        loss="mean_squared_error",
        optimizer=Adam(learning_rate=learning_rate))
    
    return model

## 3. Model Training

Finally, training the model using the single framed data and opening a tensorboard instance with details.

In [39]:
def train_model(run_name, tensorboard, model, xdata, ydata, models_folder="D:/PHD_DATA/Video_18-01-2021/models/"):
    """ Trains the given model with the given dataset. """
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
    history = model.fit(
        xdata,
        ydata,
        batch_size=2,
        validation_split=0.3,
        callbacks=[tensorboard, es],
        epochs=100
    )
    save_model(model, models_folder + run_name + ".mdl")
    return history

In [42]:
# Flexible parameters
learning_rate = 0.00001

In [ ]:
# Run each model multiple times
for i in range(0, 5):
    run_name = "SF_TEST-{}".format(int(time.time()))#"SF_32_16_8__13__222_t-{}".format(int(time.time()))
    tb = TensorBoard(log_dir=".\\logs\\{}".format(run_name))
    model = model_generator(learning_rate=learning_rate, input_shape=training_videos.shape[1:])
    history = train_model(run_name, tb, model, training_videos, training_rainfall)
    print("Final loss: {}".format(history.history["val_loss"][-1]))

Epoch 1/100
96/96 [==============================] - 61s 626ms/step - loss: 776794.0216 - val_loss: 10455.3086
Epoch 2/100
96/96 [==============================] - 56s 588ms/step - loss: 8375.2390 - val_loss: 9090.4385
Epoch 3/100
96/96 [==============================] - 55s 574ms/step - loss: 8664.1187 - val_loss: 8928.0742
Epoch 4/100
96/96 [==============================] - 49s 516ms/step - loss: 8596.5823 - val_loss: 8933.1699
Epoch 5/100
96/96 [==============================] - 53s 552ms/step - loss: 8426.2554 - val_loss: 8891.4443
Epoch 6/100
96/96 [==============================] - 55s 570ms/step - loss: 8893.7130 - val_loss: 8914.9248
Epoch 7/100
96/96 [==============================] - 52s 540ms/step - loss: 9211.3084 - val_loss: 8946.7666
Epoch 8/100
96/96 [==============================] - 51s 532ms/step - loss: 8913.9747 - val_loss: 9018.6133
Epoch 00008: early stopping
INFO:tensorflow:Assets written to: D:/PHD_DATA/Video_18-01-2021/models/SF_TEST-1612437294.mdl\assets
Fina